In [1]:
# current work directory

%pwd

'f:\\Project\\Text_Summarization\\research'

In [2]:
import os
os.chdir('../')
%pwd

'f:\\Project\\Text_Summarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    dataset_path: Path
    tokenizer_name: Path

In [29]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directories, read_yaml

class ConfigManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) ->DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            dataset_path= self.config.data_ingestion.root_dir +'/' \
                         +self.config.dataset_name+'_dataset',
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config

In [49]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset ,load_from_disk
import json

In [50]:
class DataTranformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation= True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length =128, truncation= True)
        return{
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        dataset_path = self.config.dataset_path
        dataset_dict= os.path.join(dataset_path, 'dataset_dict.json')
        save_path = os.path.join(self.config.root_dir,"samsum_dataset")

        with open(dataset_dict, 'r') as file:
            json_data = json.load(file)
            for split in json_data['splits']:
                dataset_samsum = load_from_disk(os.path.join(self.config.dataset_path, split))
                dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
                dataset_samsum_pt.save_to_disk(os.path.join(save_path, split))

In [51]:
try:
    config_manager = ConfigManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTranformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-09 21:30:24,511 : INFO : common : yaml file : config\config.yml loaded succefully]
[2024-01-09 21:30:24,517 : INFO : common : yaml file : params.yml loaded succefully]
[2024-01-09 21:30:24,520 : INFO : common : Created Director at: artifacts]
[2024-01-09 21:30:24,522 : INFO : common : Created Director at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\Omkar\anaconda3\envs\text_summerization\Lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]